<a href="https://colab.research.google.com/github/abhi789789/My-Projects/blob/master/Movie_rating.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import numpy as np
import pandas as pd
train_df=pd.read_csv("/content/gdrive/My Drive/movie_rev_Train.csv")
test_df=pd.read_csv("/content/gdrive/My Drive/MovieTest.csv")

In [0]:
xtr=train_df['review']
ytr=train_df['label']
xte=test_df['review']
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
ytr=le.fit_transform(ytr)
print(ytr)

[1 1 1 ... 0 1 1]


In [0]:
import nltk
nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import string

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
#Creating A Function To Process Data(Tokenizing,stopwards_removal,stemming)
def get_stemmed_reviw(review):
    review=review.lower()
    tokenizer=RegexpTokenizer(r'\w+')
    token=tokenizer.tokenize(review)
    en_stopwars=set(stopwords.words('english'))
    en_stopwars.remove('not')
    new_token=[token for token in token if token not in en_stopwars]
    ps=PorterStemmer()
    stem_tokn=[ps.stem(token)for token in new_token]
    clean_review=' '.join(stem_tokn)
    return(clean_review)

In [0]:
#Taking Processed Train_data in p and Processed Test_data in q
p=[get_stemmed_reviw(i) for i in xtr]
q=[get_stemmed_reviw(i) for i in xte]


In [0]:
p

In [0]:
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.layers.core import Activation,Dropout,Dense
from keras.layers.embeddings import Embedding
from keras.layers import Conv1D,LSTM,GlobalMaxPooling1D,SimpleRNN,Flatten,Bidirectional,MaxPool1D


In [0]:
#Fitting each traning word in tokenizer so as to create embedding matrix later
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(p)


In [0]:
#converting textual data in numeric form(both traning and testing data)
p= tokenizer.texts_to_sequences(p)
q= tokenizer.texts_to_sequences(q)

In [0]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

65665


In [0]:

maxlen=300
p = pad_sequences(p, padding='post', maxlen=maxlen)
q = pad_sequences(q, padding='post', maxlen=maxlen)

In [0]:
p

array([[ 1823,   638,   475, ...,     0,     0,     0],
       [ 4856,   287,  8458, ...,     0,     0,     0],
       [   69,   713,   702, ..., 11020,   153,   397],
       ...,
       [   21,   305,   460, ...,     0,     0,     0],
       [ 7666,   322,  1024, ...,     0,     0,     0],
       [  132,     2, 14622, ...,     0,     0,     0]], dtype=int32)

In [0]:
path='/content/gdrive/My Drive/GoogleNews-vectors-negative300.bin.gz'

In [0]:
#using transfer learning to train the model
import gensim
word2vec = gensim.models.KeyedVectors.load_word2vec_format(path, binary=True, limit=100000)


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
#creating a embedding matrix witht he help of google word2vec
embedding_weights = np.zeros((vocab_size, 300))
for word, index in tokenizer.word_index.items():
    #embedding_vector = word2vec.get(word)
    try:
        embedding_weights[index] = word2vec[word]
    except:
        pass 

In [0]:
embedding_weights.shape

(65665, 300)

In [0]:
print(word2vec['china'][:40])

[-0.29492188  0.08447266  0.01031494  0.34570312 -0.09375     0.05541992
  0.13183594 -0.2734375  -0.11816406  0.3125     -0.296875   -0.36914062
  0.10058594 -0.1796875  -0.16894531  0.11279297 -0.12255859  0.27539062
 -0.21582031 -0.453125    0.09472656 -0.16113281  0.3203125  -0.328125
  0.17382812  0.59765625 -0.22070312  0.19335938 -0.04467773 -0.0625
 -0.13476562 -0.10791016 -0.31445312 -0.05151367 -0.09619141 -0.15625
 -0.23828125  0.01611328  0.00613403  0.08203125]


In [0]:
from keras.callbacks import EarlyStopping 
from keras.regularizers import l2
model=Sequential()
em_layer=Embedding(vocab_size,300,weights=[embedding_weights],input_length=300,trainable=False)
model.add(em_layer)
model.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

model.summary()

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 300, 300)          19699500  
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 298, 256)          230656    
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 256)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 19,930,413
Trainable params: 230,913
Non-trainable params: 19,699,500
_________________________________________________________________


In [0]:
EarlyStopping(monitor='val_acc',
                              min_delta=0,
                              patience=1,
                              verbose=0, mode='auto')

In [0]:
history = model.fit(p, ytr, batch_size=60, epochs=15, verbose=1, validation_split=0.2)

Train on 32000 samples, validate on 8000 samples
Epoch 1/15
32000/32000 [==============================] - 10s 318us/step - loss: 0.4357 - accuracy: 0.7940 - val_loss: 0.3564 - val_accuracy: 0.8435
Epoch 2/15
32000/32000 [==============================] - 10s 308us/step - loss: 0.3306 - accuracy: 0.8566 - val_loss: 0.3253 - val_accuracy: 0.8586
Epoch 3/15
32000/32000 [==============================] - 10s 309us/step - loss: 0.2853 - accuracy: 0.8798 - val_loss: 0.3069 - val_accuracy: 0.8691
Epoch 4/15
32000/32000 [==============================] - 10s 307us/step - loss: 0.2438 - accuracy: 0.9013 - val_loss: 0.3153 - val_accuracy: 0.8609
Epoch 5/15
32000/32000 [==============================] - 10s 308us/step - loss: 0.2197 - accuracy: 0.9128 - val_loss: 0.3030 - val_accuracy: 0.8737
Epoch 6/15
32000/32000 [==============================] - 10s 306us/step - loss: 0.1871 - accuracy: 0.9276 - val_loss: 0.3012 - val_accuracy: 0.8744
Epoch 7/15
32000/32000 [==============================] -

In [0]:
ans=model.predict_classes(q)
ans=le.inverse_transform(ans)
movie=pd.DataFrame()
path=pd.read_csv("/content/gdrive/My Drive/Sample_submission.csv")
l=path['Id']
movie['Id']=l
movie['label']=ans
movie.to_csv("solution_movie5.csv",index=False)